# House sales prices in King County

A project on exploratory data analysis.

Sebastian Thomas @ neue fische Bootcamp Data Science<br />
(datascience at sebastianthomas dot de)

# Part 2: Data cleaning

We clean the data set.

## Imports

### Modules, classes and functions

In [1]:
# python object persistence
import joblib

# data
import pandas as pd

# machine learning
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

# custom modules
from modules.ds import data_type_info, cast_data_types, cast_datetime
from transformer.cleaning import clean_king_county, impute_king_county

We set the option to print float columns with two decimals.

In [2]:
pd.set_option('display.float_format', '{:,.2f}'.format)

### Data

We import our data.

In [3]:
houses_train = pd.read_pickle('data/king_county_train.pickle')
houses_train.sample(5, random_state=0)

,price,date,sqft_living,sqft_above,sqft_basement,sqft_lot,sqft_living15,sqft_lot15,bedrooms,bathrooms,floors,yr_built,yr_renovated,lat,long,zipcode,condition,grade,view,waterfront
id,,,,,,,,,,,,,,,,,,,,
8724300010,"548,000.00",9/9/2014,3420,2330,"1,090.00",5012,2320,5465,4,3.25,2.00,2008,0.00,47.73,-121.98,98019,3,10,0.00,0.00
1423400005,"249,950.00",8/15/2014,1370,1370,0.00,11658,1080,9198,3,1.00,1.00,1958,0.00,47.46,-122.18,98058,4,6,0.00,0.00
7202330280,"401,000.00",9/22/2014,1350,1350,0.00,2839,1650,3093,3,2.25,2.00,2003,0.00,47.68,-122.04,98053,3,7,0.00,0.00
1828000230,"498,000.00",7/14/2014,1620,1180,440.00,8400,2120,8424,3,2.00,1.00,1968,0.00,47.66,-122.13,98052,3,7,0.00,0.00
3342100995,"449,000.00",10/22/2014,1980,1980,0.00,5400,1980,5400,4,2.50,2.00,1998,0.00,47.52,-122.21,98056,3,8,0.00,0.00


In [4]:
data_type_info(houses_train)

,dtype,n_unique,p_unique,n_na,p_na
price,float64,3036,0.19,0,0.00
date,object,368,0.02,0,0.00
sqft_living,int64,919,0.06,0,0.00
sqft_above,int64,831,0.05,0,0.00
sqft_basement,float64,278,0.02,351,0.02
sqft_lot,int64,8038,0.50,0,0.00
sqft_living15,int64,698,0.04,0,0.00
sqft_lot15,int64,7248,0.45,0,0.00
bedrooms,int64,12,0.00,0,0.00
bathrooms,float64,29,0.00,0,0.00


## Data cleaning

We clean the features to achieve better descriptive and predictive results.

### Wrong value of feature `'bedrooms'`

The value for the feature `'bedrooms'` of instance `2402100895` is `33`. The other features, in particular `'bathrooms'`, as well as the [online entry](https://blue.kingcounty.com/Assessor/eRealProperty/Detail.aspx?ParcelNbr=2402100895) for this instance suggest that this is a typo, so we reset the value to `3`.

In [5]:
houses_train.loc[2402100895]

price           640,000.00
date             6/25/2014
sqft_living           1620
sqft_above            1040
sqft_basement       580.00
sqft_lot              6000
sqft_living15         1330
sqft_lot15            4700
bedrooms                33
bathrooms             1.75
floors                1.00
yr_built              1947
yr_renovated          0.00
lat                  47.69
long               -122.33
zipcode              98103
condition                5
grade                    7
view                  0.00
waterfront            0.00
Name: 2402100895, dtype: object

### Cleaner

We clean the dataframe `houses`, using a predefined function `clean_king_county`.

In [6]:
cleaner = FunctionTransformer(clean_king_county)

houses_train = cleaner.transform(houses_train)

## Imputing missing values

### NA values of feature `'sqft_basement'`

There are some instances whose feature `'sqft_basement'` has value `NaN`. We derive these values as differences of the values of `'sqft_living'` and `'sqft_above'`.

### NA values and wrong values of feature `'yr_renovated'`

There are some instances whose feature `'yr_renovated'` has value `0` or `NaN`. The value `0` is impossible. We interpret both values as 'the instance has never been renovated' and set the value of `'yr_renovated'` to the value of `'yr_built'` in these cases.

### NA values of feature `'view'`

There are some instances whose feature `'view'` has value `NaN`. Probably, this means that there is no view. We treat these values as `0`.

**Warning:** This approach is rough. By a comparison with the additional data that can be found in the internet (https://blue.kingcounty.com/Assessor/eRealProperty/default.aspx), we could probably justify this action or improve it.

### NA values of feature `'waterfront'`

There are some instances whose feature `'waterfront'` has value `NaN`. Probably, this means that there is no waterfront. We treat these values as `0`.

**Warning:** This approach is rough. By a comparison with the additional data that can be found in the internet (https://blue.kingcounty.com/Assessor/eRealProperty/default.aspx), we could probably justify this action or improve it. Moreover, we could perhaps enhance out proceeding by using classification on the features `'lat'` and `'long'`, or via a clustering method.

### Imputer

We impute the missing values in the dataframe `houses`, using a predefined function `impute_king_county`.

In [7]:
imputer = FunctionTransformer(impute_king_county)

houses_train = imputer.transform(houses_train)

## Casting data types

We cast the data types of the features.

In [8]:
data_type_info(houses_train)

,dtype,n_unique,p_unique,n_na,p_na
price,float64,3036,0.19,0,0.00
date,object,368,0.02,0,0.00
sqft_living,int64,919,0.06,0,0.00
sqft_above,int64,831,0.05,0,0.00
sqft_basement,float64,281,0.02,0,0.00
sqft_lot,int64,8038,0.50,0,0.00
sqft_living15,int64,698,0.04,0,0.00
sqft_lot15,int64,7248,0.45,0,0.00
bedrooms,int64,11,0.00,0,0.00
bathrooms,float64,29,0.00,0,0.00


In [9]:
caster = make_pipeline(
    FunctionTransformer(cast_data_types, kw_args={
        'int_features': ['bedrooms', 'yr_built', 'yr_renovated'],
        'float_features': ['sqft_living', 'sqft_above', 'sqft_basement', 'sqft_lot', 'sqft_living15',
                           'sqft_lot15', 'bathrooms', 'floors', 'lat', 'long'],
        'bool_features': ['waterfront'],
        'categorical_features': ['zipcode'],
        'ordered_features': ['condition', 'grade', 'view']
    }),
    FunctionTransformer(cast_datetime, kw_args={
        'datetime_features': ['date']
    })
)

houses_train = caster.transform(houses_train)

## Summary

In [10]:
houses_train.sample(5, random_state=0)

,price,date,sqft_living,sqft_above,sqft_basement,sqft_lot,sqft_living15,sqft_lot15,bedrooms,bathrooms,floors,yr_built,yr_renovated,lat,long,zipcode,condition,grade,view,waterfront
id,,,,,,,,,,,,,,,,,,,,
8724300010,"548,000.00",2014-09-09,"3,420.00","2,330.00","1,090.00","5,012.00","2,320.00","5,465.00",4,3.25,2.00,2008,2008,47.73,-121.98,98019,3,10,0,False
1423400005,"249,950.00",2014-08-15,"1,370.00","1,370.00",0.00,"11,658.00","1,080.00","9,198.00",3,1.00,1.00,1958,1958,47.46,-122.18,98058,4,6,0,False
7202330280,"401,000.00",2014-09-22,"1,350.00","1,350.00",0.00,"2,839.00","1,650.00","3,093.00",3,2.25,2.00,2003,2003,47.68,-122.04,98053,3,7,0,False
1828000230,"498,000.00",2014-07-14,"1,620.00","1,180.00",440.00,"8,400.00","2,120.00","8,424.00",3,2.00,1.00,1968,1968,47.66,-122.13,98052,3,7,0,False
3342100995,"449,000.00",2014-10-22,"1,980.00","1,980.00",0.00,"5,400.00","1,980.00","5,400.00",4,2.50,2.00,1998,1998,47.52,-122.21,98056,3,8,0,False


In [11]:
data_type_info(houses_train)

,dtype,n_unique,p_unique,n_na,p_na
price,float64,3036,0.19,0,0.00
date,datetime64[ns],368,0.02,0,0.00
sqft_living,float64,919,0.06,0,0.00
sqft_above,float64,831,0.05,0,0.00
sqft_basement,float64,281,0.02,0,0.00
sqft_lot,float64,8038,0.50,0,0.00
sqft_living15,float64,698,0.04,0,0.00
sqft_lot15,float64,7248,0.45,0,0.00
bedrooms,int64,11,0.00,0,0.00
bathrooms,float64,29,0.00,0,0.00


## Save cleaned data set

We save the cleaned data set for further analysis.

In [12]:
houses_train.to_pickle('data/king_county_train_1_cleaned.pickle')

## Save preprocessor

We construct a preprocessor object and save it for later usage.

In [13]:
preprocessor_clean = make_pipeline(cleaner, imputer, caster)

joblib.dump(preprocessor_clean, 'objects/preprocessor_clean.joblib');